In [ ]:
#Visulisation of Tourism 
# x axis time (yyyy-mm) : y axis (Tourism international visitors arrivals) + nzd-usd-exchange rate

In [10]:
install.packages("ggcorrplot")
library(tidyverse)
library(ggplot2)
library(gtable)
library(grid)
library(ggcorrplot)
library(visdat)



Installing package into ‘/home/jeffreychi/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



ERROR: Error in library(timeDate): there is no package called ‘timeDate’


In [22]:
#Fetch all merged data
data_tourisms <- read_csv("data/tourism_final_clean_arriavals_1921Q1-2022Q2.csv", show_col_types = FALSE)

df_nz_tourism <- data.frame(data_tourisms)
colnames(df_nz_tourism)<-c('Id','date.quarter','Actual_counts')
#ata_tourisms %>% glimpse()
head(df_nz_tourism)
#vis_miss(data_tourisms)

New names:
• `` -> `...1`


,Id,date.quarter,Actual_counts
,<dbl>,<chr>,<dbl>
1,1,1921-2,2934
2,2,1921-3,1808
3,3,1921-4,3783
4,4,1922-1,3801
5,5,1922-2,1964
6,6,1922-3,1565


In [24]:
#Fetch NZD VS. USD exchange rate history
# Normalization for each time formats
install.packages("timeDate")
install.packages("here")
library(timeDate)
library(tidyverse)
library(visdat)
library(here)
library(readxl)

# convert "1999Q1" to "1999-03"
Yq2Ym <- function(date) {
           dt = str_split(date, "Q")
           y = dt[[1]][[1]]
           q = strtoi(dt[[1]][[2]], 10)
           m = q * 3
           mm = str_pad(m, 2, side = "left", pad = 0)
           ym = paste(y, mm, sep = "-")
           ym
       }

# convert "1999-3-29" to "1999-03"
Yqd2Ym <- function(date) {
           dt = str_split(date, "-")
           y = dt[[1]][[1]]
           m = strtoi(dt[[1]][[2]], 10)
           mm = str_pad(m, 2, side = "left", pad = 0)
           ym = paste(y, mm, sep = "-")
           ym
       }

# Convert "1999Q1" to the last day of quarter
Yq2Date <- function(date) {
           ym = Yq2Ym(date)
           ymd = paste(ym, "01", sep = "-")
           dt = as.Date(timeLastDayInMonth(ymd))
           dt
       }


# Convert "1999-03-29" to the last day of month
Yqd2Date <- function(date) {
          ym = Yqd2Ym(date)
          ymd = paste(ym, "01", sep = "-")
          dt = as.Date(timeLastDayInMonth(ymd))
          dt
       }

# convert "1999-03" to "1999Q1"
Ym2Yq <- function(date) {
           dt = str_split(date, "-")
           y = dt[[1]][[1]]
           m = strtoi(dt[[1]][[2]], 10)
           q = (m - 1) %/% 3 + 1
           yq = paste(y, q, sep = "Q")
           yq
       }

# exchange rate NZD-USD
nzd_usd <- read_csv("data/nzd-usd_exchange_rate.csv", col_select = c("date", "USD exchange rate"), show_col_types = FALSE)
nzd_usd$date.month = map_chr(nzd_usd$date, Yqd2Ym)
nzd_usd$date.datetime = map(nzd_usd$date, Yqd2Date)

m_nzd_usd <- nzd_usd %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01"))

m_nzd_usd$date.quarter = map_chr(m_nzd_usd$date.month, Ym2Yq)
# m_nzd_usd$usd_rate_m = m_nzd_usd$'USD exchange rate'
names(m_nzd_usd)[names(m_nzd_usd) == "USD exchange rate"] <- "usd_rate_m"

q_nzd_usd <- m_nzd_usd %>%
    group_by(date.quarter) %>%
    summarise(usd_rete_quarter = mean(usd_rate_m)) %>%
    mutate(date.quarter = map_chr(.$date.quarter, Yq2Ym))
colnames(q_nzd_usd)<-c('date.quarter','usd_rate_quarter')
q_nzd_usd %>% head(10)

Installing package into ‘/home/jeffreychi/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jeffreychi/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



date.quarter,usd_rate_quarter
<chr>,<dbl>
1999-03,0.5383333
1999-06,0.5426667
1999-09,0.5253333
1999-12,0.5120000
2000-03,0.4986667
2000-06,0.4793333
2000-09,0.4420000
2000-12,0.4093333
2001-03,0.4333333


In [25]:
#Merge tourisms arrivals data and nzd_usd_exchange_rate

merge(df_nz_tourism, q_nzd_usd, by="data.quarter") 

ERROR: Error in fix.by(by.x, x): 'by' must specify a uniquely valid column


In [ ]:
#Plotting

p1 <- ggplot(data, aes(x = date.quarter, y =  ump_rate, group = 1)) +
    geom_line(aes(col = "Unemployment")) +
    geom_bar(aes(y = ump_rate, fill = "Unemployment"), stat = "identity", alpha=.3) +
    geom_line(aes(y = ocr_quarter, col = "Official Cash Rate")) +
    scale_fill_manual(values = c("Unemployment" = "dodgerblue3")) +
    scale_x_discrete(name = "Time(Quarterly)", breaks = x_breaks) +
    scale_y_continuous(name = "Unemployment Rate",
        # Add a second axis and specify its features
        sec.axis = sec_axis(~., name = "Official Cash Rate")) + 
    scale_color_manual(values = c("Unemployment" = "dodgerblue3", "Official Cash Rate" = "darkorange")) +
    theme(legend.position="bottom", plot.margin=grid::unit(c(0,5,0,5), "mm")) +
    theme(axis.text.x = element_text(angle = -45, hjust = 0.001)) + 
    coord_fixed(ratio = 6) +
    labs(color = " ", fill = " ", 
       title = paste0("Unemployment Rate ~ Official Cash Rate\nDate: ", Sys.Date()))
p1